### Warm-ups

1. **In-place or copy?** — Return a **new** list with x appended; don’t mutate input.

In [1]:
def append_copy(xs, x):
    """Return new list which is xs with x appended."""
    return xs + [x]

a = [1,2]; b = append_copy(a, 3)
assert a == [1,2] and b == [1,2,3]


2. **Safe default parameter**

In [2]:
def append_safe(x, bucket=None):
    """Append x to bucket but avoid shared default list."""
    if bucket is None:
        bucket = []
    bucket.append(x)
    return bucket

b1 = append_safe(1); b2 = append_safe(2)
assert b1 == [1] and b2 == [2]

3. **Clone 2-D list (shallow vs deep)**

In [3]:

def shallow_clone_2d(m): 
    """Return shallow copy of 2-D list m."""
    return [row for row in m]

def deep_clone_2d(m): 
    """Return deep copy of 2-D list m."""
    return [[x for x in row] for row in m]

m = [[1],[2]]
m2 = shallow_clone_2d(m); m3 = deep_clone_2d(m)
m[0].append(9)
assert m2[0] == [1,9] and m3[0] == [1]

### Core

4. **Freeze nested** (list/tuple/dict → immutables)

In [4]:
def freeze(obj):
    """Convert lists to tuples, dicts to tuples of (key, frozen(value)) sorted by key."""
    if isinstance(obj, list):
        return tuple(freeze(x) for x in obj)
    elif isinstance(obj, dict):
        return tuple((k, freeze(v)) for k, v in sorted(obj.items()))
    else:
        return obj
    
f = freeze({"a":[1,2], "b":{"x":1}})
assert isinstance(f, tuple) and ("a", (1,2)) in f

5. **Deep update (write-on-copy)**
   Merge dict `updates` into dict `base` without mutating `base`; nested dicts should merge recursively.

In [5]:
def deep_merge(base, updates):
    """Return new dict with updates merged into base recursively."""
    result = {}
    for k in base.keys() | updates.keys():
        if k in base and k in updates:
            if isinstance(base[k], dict) and isinstance(updates[k], dict):
                result[k] = deep_merge(base[k], updates[k])
            else:
                result[k] = updates[k]
        elif k in base:
            result[k] = base[k]
        else:
            result[k] = updates[k]
    return result

base = {"a":1,"b":{"x":1}}
u = {"b":{"y":2},"c":3}
merged = deep_merge(base,u)
assert base == {"a":1,"b":{"x":1}}
assert merged == {"a":1,"b":{"x":1,"y":2},"c":3}

6. **Sort safely** — Return a sorted copy by key; original list of dicts stays unchanged.

In [6]:
def sort_users_by(users, key):
    """Return new list of users sorted by given key."""
    return sorted(users, key=lambda u: u[key])

U = [{"name":"b","age":30},{"name":"a","age":20}]
out = sort_users_by(U,"name")
assert out[0]["name"]=="a" and U[0]["name"]=="b"

7. **Toggle flag in dict** — don’t mutate input

In [7]:
def toggled(d, key):
    """Return a copy with boolean d[key] toggled (missing -> True)."""
    new_d = d.copy()
    new_d[key] = not d.get(key, False)
    return new_d

assert toggled({"a":False},"a") == {"a":True}

### Challenge

8. **deep\_copy\_except(keys\_to\_skip)**
   Deep-copy a nested structure but **preserve references** for subtrees whose top-level keys are in `keys_to_skip` (for dicts).

In [8]:
def deep_copy_except(obj, keys_to_skip=frozenset()):
    """Deep copy obj except for keys in keys_to_skip."""
    if isinstance(obj, dict):
        return {k: (obj[k] if k in keys_to_skip else deep_copy_except(obj[k], keys_to_skip)) for k in obj}
    elif isinstance(obj, list):
        return [deep_copy_except(x, keys_to_skip) for x in obj]
    else:
        return obj
    
src = {"cfg":{"x":1}, "cache":[1,2]}
out = deep_copy_except(src, {"cache"})
assert out["cfg"] is not src["cfg"] and out["cache"] is src["cache"]